In [1]:
from flask import Flask, request, jsonify

from transformers import AutoTokenizer
import torch
from flask import Flask
from flask_cors import CORS
from modeling import create_model
from scaler import Scaler, load_pretrained_scaler
from inference_api import transform_input, predict_price

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_file = "t5config.json"
pretrained_base_model_name = "VietAI/vit5-base"

In [3]:
from asyncio.windows_events import NULL
from email.mime import image
import os
import secrets
from turtle import title
from unittest import result
from PIL import Image
from flask import Flask, render_template, url_for, flash, redirect
from form import PictureForm

app = Flask(__name__)
app.config['SECRET_KEY'] = '7d0fb494ff22bc777a2384efae5670f5b6c41fafb4738df9cd4f50741d15dd78'
print(app.root_path)
print(os.path.join(app.root_path.removesuffix('\FlaskWeb'), '', '123.txt'))

CORS(app)
use_gpu = False
# Khởi tạo mô hình
device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
model = create_model(config_file, state_dict_path="checkpoint/model_state_dict.pt", device=device)
model.eval()

# Khởi tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_base_model_name)

# Load Scaler
scaler = load_pretrained_scaler("scaler.pt")
# Cấu hình CORS với tùy chọn cụ thể
def saveinput(form_input):
    random_hex = secrets.token_hex(8)
    _, f_ext = os.path.splitext(form_input.filename)
    picture_fn = random_hex + f_ext
    picture_path = os.path.join(os.path.join(app.root_path, 'static/images', picture_fn))
    # os.path.join(app.root_path, 'static/images', picture_fn)

    # output_size = (125, 125)
    i = Image.open(form_input)
    # i.thumbnail(output_size)
    i.save(picture_path)

    return picture_fn

@app.route('/', methods=['GET', 'POST'])
@app.route('/home', methods=['GET', 'POST'])
@app.route('/predict', methods=['POST'])
def home():
    form = PictureForm()
    image_file = url_for('static', filename='images/webpc.png')
    if form.validate_on_submit():
        if form.picture.data:
            picture_file = saveinput(form.picture.data)
            image_file = url_for('static', filename='images/' + picture_file)
    result = "10"
    return render_template('home.html', image_file=image_file, title='Home', form=form, result=result)

def predict():
    data = request.get_json()
    print(data)
    # data: {"estate_type", "square", "description"}

    # Xử lý dữ liệu đầu vào
    input_encoding = transform_input(tokenizer, data)

    # Dự đoán giá
    price = predict_price(model, scaler, input_encoding)

    # return jsonify({'price': price})
    return render_template('home.html', price=price, title='Home' )

if __name__ == '__main__':
    app.run(host='localhost', port='5000', debug=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3002
Press CTRL+C to quit
127.0.0.1 - - [17/Dec/2023 17:18:45] "OPTIONS /predict HTTP/1.1" 200 -
